In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import ascii
from collections import Counter
import time
rcParams['font.family'] = 'serif'
% matplotlib

Using matplotlib backend: TkAgg


In [2]:
# Reading of the data
data = np.load('DL07_milimill_HOD_BIN.npy')
IDs = data[:,0]
fofid = (data[:,1]).astype(np.int64)
stellarmass = data[:,2]
centralmvir = np.log10(data[:,3] + 1e-10) + 10
Type = data[:,4]

In [3]:
log_centralmvir = centralmvir
IDS, index = np.unique(fofid, return_index = True)
halo_masses = log_centralmvir[index]

NBIN = 30
logm_min, logm_max = 10, 14.
mass_labels = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)

bin_width = (logm_max - logm_min)/NBIN

#bins = np.linspace(logm_min, logm_max, NBIN) + bin_width *.5
bins = np.array([10 + bin_width*i for i in range(NBIN)]) + bin_width *.5

In [4]:
def counter (T, F, M): 
    Halos = len(T)
    Ngal, Ncen, Nsat = 0, 0, 0
    Halo_data = np.zeros((Halos, 5))
    for i in range(Halos):
        
        T_array = T[i]
        M_array = M[i]
        F_array = F[i]
        
        mass_mask = M_array > .1
        T_array = T_array[mass_mask]
        F_array = F_array[mass_mask]
        
        sat_mask = T_array != 0
        Nsat += len(T_array[sat_mask])
        Ncen += int(np.any(T_array == 0))
        Ngal += len(T_array)
        
        if len(F_array) == 0: continue
        pos = np.where(F_array[0] == IDS)
        Hmass = halo_masses[pos]
        Halo_data[i] = F_array[0], Ngal, Ncen, Nsat, Hmass
        
    return Halos, Ngal, Ncen, Nsat, Halo_data
 
def ToList (fofid, Type, stellarmass):
    
    fofid_list = []
    type_list = []
    mass_list = []
    C = Counter(fofid)
    
    for FOFID in C.keys():
        
        mask_fofid = fofid == FOFID
        temp_fofid = fofid[mask_fofid]
        temp_Type = Type[mask_fofid]
        temp_stellarmass = stellarmass[mask_fofid]
        
        type_list.append(temp_Type)
        fofid_list.append(temp_fofid)  
        mass_list.append(temp_stellarmass)

    return fofid_list, type_list, mass_list

In [5]:
from collections import Counter
a, idx = np.unique(fofid, return_index = True)

Halos = np.zeros(NBIN)
Galaxies = np.zeros(NBIN)
Cent = np.zeros(NBIN)
Sats = np.zeros(NBIN)

for i in range(NBIN):
    
    # Índices correspondientes al bin de masa
    idx = np.where(mass_labels == i)[0]

    # Seleccionamos sólo los valores que corresponden al bin
    bin_halomasses = log_centralmvir[idx] 
    bin_stellarmass = stellarmass[idx]
    bin_fofid = fofid[idx]
    bin_type = Type[idx]

    FOFID_list, TYPE_list, MASS_list = ToList(bin_fofid, bin_type, bin_stellarmass)
    
    if i > 0:
        Halos[i], Galaxies[i], Cent[i], Sats[i], temp_DATA = counter(TYPE_list, FOFID_list, MASS_list)
        DATA = np.append(DATA, temp_DATA, axis = 0)
        continue
        
    # HOD, Centrals y Sats cumplen la condición de MASA
    Halos[i], Galaxies[i], Cent[i], Sats[i], DATA = counter(TYPE_list, FOFID_list, MASS_list)
    
idx = np.where(DATA[:,0] == 0)[0]
DATA = np.delete(DATA, idx, axis = 0)
np.save("HOD_data", DATA)

In [ ]:
#Galaxies += 1e-10
Cent += 1e-10
Sats += 1e-10
HOD = Galaxies/Halos
Cent /= Halos
Sats /= Halos

In [ ]:
bins2, HOD2, cent, sats = np.loadtxt("HOD.txt", unpack = True)

In [ ]:
plt.plot(bins, np.log10(HOD), 'k.', label = 'All')
plt.plot(bins, np.log10(Cent), 'r--', label = 'Centrals')
plt.plot(bins, np.log10(Sats), 'b--', label = 'Sats')
#plt.plot(bins2, np.log10(HOD2), 'y.')  # To Compare with first method
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$")
plt.ylabel(r"$log(N)$")
plt.ylim(-2,2)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left', frameon = False, prop={'size':10})
plt.show()